In [5]:
import requests
import os
import pytest
import requests
import json
import datetime
from datetime import datetime, timedelta
from email.utils import formatdate
import certifi
import pandas as pd
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

SyntaxError: expected 'except' or 'finally' block (55632668.py, line 143)

In [ ]:
# Get today's date
today_date = datetime.date.today()

# Convert to the desired format
formatted_date = today_date.strftime('%Y-%m-%d')

# API base URL
base_url = "https://sandboxapi.psd.dnb.no"

# Define headers and body for creating a consent
headers_create_consent = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "X-Request-ID": "2ff74d04-d0ad-4d70-a945-c5e3e5ab9b90", 
    "TPP-Redirect-Preferred": "true",
    "TPP-Redirect-URI": "http://dnb.no",
    "PSU-ID": "31125453913",  
    "PSU-IP-Address": "80.213.0.32 "
}

body_create_consent = {
    "validUntil": formatted_date,
    "frequencyPerDay": 1,
    "access": {
        "balances": [],
        "accounts": [],
        "transactions": []
    },
    "recurringIndicator": True,
    "combinedServiceIndicator": False
}

response_create_consent = requests.post(
    base_url + "/v2/consents", 
    headers=headers_create_consent, 
    data=json.dumps(body_create_consent),
    cert=('certificates/certificate.pem', 'certificates/private.key')
)

response_data = response_create_consent.json()

response_data


In [ ]:
# Assuming the consent was created successfully, we retrieve the consent ID
consent_id = response_data['consentId']

# Define headers for getting account information
headers_get_accounts = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "X-Request-ID": "2ff74d04-d0ad-4d70-a945-c5e3e5ab9b90",
    "TPP-Redirect-Preferred": "true",
    "TPP-Redirect-URI": "http://dnb.no",
    "Consent-ID": consent_id,
    "PSU-IP-Address": "80.213.0.32"
}

response_accounts = requests.get(
    base_url + "/v2/accounts", 
    headers=headers_get_accounts, 
    cert=('certificates/certificate.pem', 'certificates/private.key')
)
accounts_data = response_accounts.json()

accounts_data


In [ ]:
account_balances = {}

for account in accounts_data['accounts']:
    balance_endpoint = account['_links']['balances']['href']
    response_balance = requests.get(
        base_url + balance_endpoint, 
        headers=headers_get_accounts, 
        cert=('certificates/certificate.pem', 'certificates/private.key')
    )
    balance_data = response_balance.json()

    # Check if the keys exist before attempting to access them
    if 'balances' in balance_data and len(balance_data['balances']) > 0:
        if 'balanceAmount' in balance_data['balances'][0] and 'amount' in balance_data['balances'][0]['balanceAmount']:
            key = account['name'] + " - " + account['iban']  # Combining name with IBAN for uniqueness
            account_balances[key] = balance_data['balances'][0]['balanceAmount']['amount']

# Print the balances
for account_name, balance in account_balances.items():
    print(f"{account_name}: {balance} NOK")

In [ ]:
# Checking if the accounts have transactions

account_transactions = {}

for account in accounts_data['accounts']:
    transaction_endpoint = account['_links']['transactions']['href'] + "?bookingStatus=booked"  # Added bookingStatus parameter
    
    response_transactions = requests.get(
        base_url + transaction_endpoint,
        headers=headers_get_accounts,
        cert=('certificates/certificate.pem', 'certificates/private.key')
    )
    
    # Debugging: Check HTTP status and full response
    print(f"HTTP Status Code for transactions: {response_transactions.status_code}")
    print(f"Full Response for transactions: {response_transactions.json()}")

    transactions_data = response_transactions.json()
    
    # Debugging: Check the structure of transactions_data
    print(f"Transactions Data: {transactions_data}")

    # Assuming transactions are stored under a key named 'transactions'
    if 'transactions' in transactions_data:
        key = account['name'] + " - " + account['iban']  # Combining name with IBAN for uniqueness
        account_transactions[key] = transactions_data['transactions']
        
# Print the transactions
for account_name, transactions in account_transactions.items():
    print(f"Transactions for {account_name}:")
    
    booked_transactions = transactions.get('booked', [])
    
    if not booked_transactions:
        print("  No transactions available.")
        continue
    
    for transaction in booked_transactions:
        # Assuming transaction is a dictionary containing at least an 'amount' field.
        # Replace with the correct key names based on your actual API response.
        print(f"  Amount: {transaction['amount']} NOK, Description: {transaction.get('description', 'N/A')}")
